In [2]:
import requests
from bs4 import BeautifulSoup

#fetch HTML code
r = requests.get("http://www.pyclass.com/real-estate/rock-springs-wy/LCWYROCKSPRINGS/", headers={'User-agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:61.0) Gecko/20100101 Firefox/61.0'})
c = r.content

# parse requested content in HTML format using BeautifulSoup
soup = BeautifulSoup(c,"html.parser") 
#print(soup.prettify()) # to test that HTML script has been read correctly

#from inspecting HTML, we know that each listing is a div element with class propertyRow
all = soup.find_all("div",{"class":"propertyRow"})

#extract price of first listing from h4 tag with class propPrice
all[0].find("h4",{"class":"propPrice"}).text.replace("\n","").replace(" ","") # convert 'Tag' to text, remove \n and spaces